# limpiesa y lectura de datos


In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import re

DATA_DIR = Path("csv").resolve()
assert DATA_DIR.exists(), f"No existe {DATA_DIR}"

# Detectar archivos
asistencia_files = sorted(DATA_DIR.glob("Asistencia_Anual_*.csv"))
rend_files = sorted(DATA_DIR.glob("Rendimiento_*.csv"))

print("Rendimiento:", [f.name for f in rend_files])

# Helper robusto de lectura
def read_csv_smart(path: Path, nrows=None):
    try:
        return pd.read_csv(path, nrows=nrows)
    except UnicodeDecodeError:
        return pd.read_csv(path, nrows=nrows, encoding="latin-1")

Rendimiento: ['Rendimiento_2022_2023.csv', 'Rendimiento_2023_2024.csv', 'Rendimiento_2024_2025.csv']


In [3]:
def read_csv_smart(path: Path, nrows=None):
    """
    Lee CSV detectando separador y encoding. Omite líneas defectuosas.
    """
    for enc in ("utf-8", "utf-8-sig", "latin-1"):
        # 1) auto-inferencia de separador (engine=python)
        try:
            return pd.read_csv(path, nrows=nrows, sep=None, engine="python",
                               encoding=enc, on_bad_lines="skip")
        except UnicodeDecodeError:
            continue
        except pd.errors.ParserError:
            pass
        # 2) intentos explícitos de separador común
        for sep in (";", "\t", "|", ","):
            try:
                return pd.read_csv(path, nrows=nrows, sep=sep, engine="python",
                                   encoding=enc, on_bad_lines="skip")
            except Exception:
                continue
    # 3) último recurso
    return pd.read_csv(path, nrows=nrows, sep=";", engine="python",
                       encoding="latin-1", on_bad_lines="skip")

# Inspección rápida de columnas
for f in asistencia_files + rend_files:
    dfh = read_csv_smart(f, nrows=5)
    print(f"\n{f.name} -> {dfh.shape}")
    print(dfh.columns.tolist())
    display(dfh.head())

# Hints para mapear nombres
COLUMN_HINTS = {
    "id": ["id", "id_alumno", "idalumno", "rut", "estudiante", "id_estudiante"],
    "anio": ["anio", "año", "ano", "periodo", "year"],
    "asistencia": ["asistencia", "asistencia_%", "porc_asistencia", "asistencia_pct"],
    "nota": ["promedio", "nota_final", "nota", "gpa"],
    "aprobado": ["aprobado", "estado", "resultado"]
}

def infer_col(df, keys):
    cols = [c for c in df.columns]
    low = {c.lower(): c for c in cols}
    for k in keys:
        for lc, orig in low.items():
            if re.search(rf"\b{k}\b", lc):
                return orig
    return None


Rendimiento_2022_2023.csv -> (5, 37)
['\ufeffAGNO', 'RBD', 'DGV_RBD', 'NOM_RBD', 'COD_REG_RBD', 'NOM_REG_RBD_A', 'COD_PRO_RBD', 'COD_COM_RBD', 'NOM_COM_RBD', 'COD_DEPROV_RBD', 'NOM_DEPROV_RBD', 'COD_DEPE', 'COD_DEPE2', 'RURAL_RBD', 'ESTADO_ESTAB', 'COD_ENSE', 'COD_ENSE2', 'COD_GRADO', 'LET_CUR', 'COD_JOR', 'COD_TIP_CUR', 'COD_DES_CUR', 'MRUN', 'GEN_ALU', 'FEC_NAC_ALU', 'EDAD_ALU', 'COD_REG_ALU', 'COD_COM_ALU', 'NOM_COM_ALU', 'COD_RAMA', 'COD_SEC', 'COD_ESPE', 'PROM_GRAL', 'ASISTENCIA', 'SIT_FIN', 'SIT_FIN_R', 'COD_MEN']


,﻿AGNO,RBD,DGV_RBD,NOM_RBD,COD_REG_RBD,NOM_REG_RBD_A,COD_PRO_RBD,COD_COM_RBD,NOM_COM_RBD,COD_DEPROV_RBD,...,COD_COM_ALU,NOM_COM_ALU,COD_RAMA,COD_SEC,COD_ESPE,PROM_GRAL,ASISTENCIA,SIT_FIN,SIT_FIN_R,COD_MEN
0,2022,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,"6,6",98,P,P,0
1,2022,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,0,0,Y,T,0
2,2022,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,0,0,Y,Y,0
3,2022,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,0,0,Y,T,0
4,2022,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,0,0,Y,Y,0



Rendimiento_2023_2024.csv -> (5, 37)
['\ufeffAGNO', 'RBD', 'DGV_RBD', 'NOM_RBD', 'COD_REG_RBD', 'NOM_REG_RBD_A', 'COD_PRO_RBD', 'COD_COM_RBD', 'NOM_COM_RBD', 'COD_DEPROV_RBD', 'NOM_DEPROV_RBD', 'COD_DEPE', 'COD_DEPE2', 'RURAL_RBD', 'ESTADO_ESTAB', 'COD_ENSE', 'COD_ENSE2', 'COD_GRADO', 'LET_CUR', 'COD_JOR', 'COD_TIP_CUR', 'COD_DES_CUR', 'MRUN', 'GEN_ALU', 'FEC_NAC_ALU', 'EDAD_ALU', 'COD_REG_ALU', 'COD_COM_ALU', 'NOM_COM_ALU', 'COD_RAMA', 'COD_SEC', 'COD_ESPE', 'PROM_GRAL', 'ASISTENCIA', 'SIT_FIN', 'SIT_FIN_R', 'COD_MEN']


,﻿AGNO,RBD,DGV_RBD,NOM_RBD,COD_REG_RBD,NOM_REG_RBD_A,COD_PRO_RBD,COD_COM_RBD,NOM_COM_RBD,COD_DEPROV_RBD,...,COD_COM_ALU,NOM_COM_ALU,COD_RAMA,COD_SEC,COD_ESPE,PROM_GRAL,ASISTENCIA,SIT_FIN,SIT_FIN_R,COD_MEN
0,2023,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,0,0,Y,Y,0
1,2023,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,"5,7",94,P,P,0
2,2023,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,0,0,Y,Y,0
3,2023,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,"6,7",100,P,P,0
4,2023,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,0,0,Y,Y,0



Rendimiento_2024_2025.csv -> (5, 38)
['\ufeffAGNO', 'RBD', 'DGV_RBD', 'NOM_RBD', 'COD_REG_RBD', 'NOM_REG_RBD_A', 'COD_PRO_RBD', 'COD_COM_RBD', 'NOM_COM_RBD', 'COD_DEPROV_RBD', 'NOM_DEPROV_RBD', 'COD_DEPE', 'COD_DEPE2', 'RURAL_RBD', 'ESTADO_ESTAB', 'NOMBRE_SLEP', 'COD_ENSE', 'COD_ENSE2', 'COD_GRADO', 'LET_CUR', 'COD_JOR', 'COD_TIP_CUR', 'COD_DES_CUR', 'MRUN', 'GEN_ALU', 'FEC_NAC_ALU', 'EDAD_ALU', 'COD_REG_ALU', 'COD_COM_ALU', 'NOM_COM_ALU', 'COD_RAMA', 'COD_SEC', 'COD_ESPE', 'PROM_GRAL', 'ASISTENCIA', 'SIT_FIN', 'SIT_FIN_R', 'COD_MEN']


,﻿AGNO,RBD,DGV_RBD,NOM_RBD,COD_REG_RBD,NOM_REG_RBD_A,COD_PRO_RBD,COD_COM_RBD,NOM_COM_RBD,COD_DEPROV_RBD,...,COD_COM_ALU,NOM_COM_ALU,COD_RAMA,COD_SEC,COD_ESPE,PROM_GRAL,ASISTENCIA,SIT_FIN,SIT_FIN_R,COD_MEN
0,2024,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,"6,2",99,P,P,0
1,2024,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,0,0,Y,T,0
2,2024,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,"5,2",85,R,R,0
3,2024,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,"5,8",98,P,P,0
4,2024,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,0,0,Y,T,0


In [ ]:
# ...existing code...

# ================== LIMPIEZA FOCAL SOLO RENDIMIENTO ==================
from pathlib import Path
import pandas as pd, numpy as np

OUT_DIR = Path("csvClear")
OUT_DIR.mkdir(exist_ok=True)

REQUIRED_REND = [
    "AGNO","MRUN","PROM_GRAL","ASISTENCIA","SIT_FIN","SIT_FIN_R"
]

for f in rend_files:
    
   
    # leer, seleccionar sólo las columnas necesarias, limpiar y guardar
    for f in rend_files:
        df = read_csv_smart(f)
        # normalizar nombres de columna (quitar BOM, espacios, pasar a mayúsculas)
        cols = [str(c).strip().lstrip("\ufeff").upper() for c in df.columns]
        df.columns = cols

        # seleccionar columnas disponibles de REQUIRED_REND
        present = [c for c in REQUIRED_REND if c in df.columns]
        missing = [c for c in REQUIRED_REND if c not in df.columns]
        if missing:
            print(f"{f.name}: faltan columnas {missing} -> se guardarán sólo {present}")

        DFIni = df[present].copy()

        # limpiezas sencillas: numeros con coma, porcentajes, tipos
        if "PROM_GRAL" in DFIni.columns:
            DFIni["PROM_GRAL"] = DFIni["PROM_GRAL"].astype(str).str.replace(",", ".").replace({"nan": None})
            DFIni["PROM_GRAL"] = pd.to_numeric(DFIni["PROM_GRAL"], errors="coerce")

        if "ASISTENCIA" in DFIni.columns:
            DFIni["ASISTENCIA"] = DFIni["ASISTENCIA"].astype(str).str.replace(",", ".").str.replace("%", "")
            DFIni["ASISTENCIA"] = pd.to_numeric(DFIni["ASISTENCIA"], errors="coerce")

        # opcional: normalizar SIT_FIN como cadena limpia
        for c in ("SIT_FIN", "SIT_FIN_R"):
            if c in DFIni.columns:
                DFIni[c] = DFIni[c].astype(str).str.strip().replace({"nan": None})

        out_path = OUT_DIR / f.name.replace(".csv", "_clean.csv")
        DFIni.to_csv(out_path, index=False)
        print("Guardado:", out_path)

Guardado: csvClear/Rendimiento_2022_2023_clean.csv
Guardado: csvClear/Rendimiento_2023_2024_clean.csv
Guardado: csvClear/Rendimiento_2024_2025_clean.csv
Guardado: csvClear/Rendimiento_2022_2023_clean.csv
Guardado: csvClear/Rendimiento_2023_2024_clean.csv
Guardado: csvClear/Rendimiento_2024_2025_clean.csv
Guardado: csvClear/Rendimiento_2022_2023_clean.csv


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

def load_rendimiento_clean(data_dir='./datasets/csvClear'):
    """
    Carga todos los Rendimiento_*_clean.csv generados por datasets/leercsv.ipynb,
    normaliza tipos y columnas básicas.
    """
    base = Path(data_dir)
    files = sorted(base.glob('Rendimiento_*_clean.csv'))
    if not files:
        raise FileNotFoundError(f"No se encontraron archivos *_clean en {base}. Corre primero datasets/leercsv.ipynb para generar csvClear/")
    
    dfs = []
    for f in files:
        df = pd.read_csv(f)
        # Normalizar nombres
        df.columns = [str(c).strip().upper().lstrip('\ufeff') for c in df.columns]
        # Asegurar columnas clave
        for c in ["AGNO","MRUN","PROM_GRAL","ASISTENCIA","SIT_FIN","SIT_FIN_R"]:
            if c not in df.columns:
                df[c] = np.nan
        # Tipos
        df["AGNO"] = pd.to_numeric(df["AGNO"], errors="coerce").astype("Int64")
        df["MRUN"] = df["MRUN"].astype(str).str.strip()
        # decimales con coma ya vienen limpios desde csvClear; reforzamos
        df["PROM_GRAL"] = pd.to_numeric(df["PROM_GRAL"], errors="coerce")
        df["ASISTENCIA"] = pd.to_numeric(df["ASISTENCIA"], errors="coerce")
        # strings
        for c in ("SIT_FIN","SIT_FIN_R"):
            df[c] = df[c].astype(str).str.strip().replace({"nan": np.nan, "None": np.nan})
        df["SOURCE_FILE"] = f.name
        dfs.append(df)
    full = pd.concat(dfs, ignore_index=True)

    # Normalizar asistencia a 0-100
    # Si hay valores en [0,1], escalamos a %; si hay >100, los truncamos
    mask_01 = full["ASISTENCIA"].between(0, 1, inclusive="both")
    full.loc[mask_01, "ASISTENCIA"] = full.loc[mask_01, "ASISTENCIA"] * 100
    full["ASISTENCIA"] = full["ASISTENCIA"].clip(lower=0, upper=100)

    # Quitar MRUN vacíos
    full = full[full["MRUN"].notna() & (full["MRUN"].str.len() > 0)].copy()
    # Deduplicar por AGNO+MRUN conservando último
    full = full.sort_values(["AGNO","MRUN"]).drop_duplicates(["AGNO","MRUN"], keep="last")
    print(f"✅ Rendimiento cargado: {len(full):,} filas de {len(files)} archivos")
    print(full[["AGNO","PROM_GRAL","ASISTENCIA","SIT_FIN","SIT_FIN_R"]].head())
    return full

def build_label_aprobacion(df: pd.DataFrame) -> pd.DataFrame:
    """
    Crea label binario 'label_aprobado' a partir de SIT_FIN_R / SIT_FIN y, si falta, por regla PROM_GRAL>=4.0.
    1 = Aprobado/Promovido, 0 = Reprobado/Retiro/Deserción/etc.
    """
    df = df.copy()
    y = pd.Series(np.nan, index=df.index, dtype="float")

    # Fuente 1: SIT_FIN_R
    s = df["SIT_FIN_R"].fillna("").str.lower()
    aprob = s.str.contains(r"apro|promov") & ~s.str.contains(r"no\s*apro")
    reprob = s.str.contains(r"reprob|repit|retir|deser|elim|baja|aband")
    y.loc[aprob] = 1
    y.loc[reprob] = 0

    # Fuente 2: SIT_FIN (fallback)
    s2 = df["SIT_FIN"].fillna("").str.lower()
    aprob2 = s2.str.contains(r"apro|promov") & ~s2.str.contains(r"no\s*apro")
    reprob2 = s2.str.contains(r"reprob|repit|retir|deser|elim|baja|aband")
    y.loc[y.isna() & aprob2] = 1
    y.loc[y.isna() & reprob2] = 0

    # Fuente 3: Regla por nota
    y.loc[y.isna() & (pd.to_numeric(df["PROM_GRAL"], errors="coerce") >= 4.0)] = 1
    y.loc[y.isna() & (pd.to_numeric(df["PROM_GRAL"], errors="coerce") < 4.0)] = 0

    # Limpieza final
    df["label_aprobado"] = y.astype("Int64")
    vc = df["label_aprobado"].value_counts(dropna=False).to_dict()
    n_na = int(df["label_aprobado"].isna().sum())
    print(f"✅ Label creado: {vc} | sin definir: {n_na}")
    return df

def engineer_features_edu(df: pd.DataFrame) -> pd.DataFrame:
    """
    Features básicas: PROM_GRAL, ASISTENCIA(0-1), AGNO one-hot opcional.
    """
    df = df.copy()
    df["prom_gral"] = pd.to_numeric(df["PROM_GRAL"], errors="coerce")
    df["asistencia_pct"] = pd.to_numeric(df["ASISTENCIA"], errors="coerce")/100.0
    # Opcional: bucket de año
    df["agno"] = pd.to_numeric(df["AGNO"], errors="coerce").astype("Int64")
    # Imputaciones simples
    df["prom_gral"] = df["prom_gral"].clip(lower=1.0, upper=7.0)
    df["asistencia_pct"] = df["asistencia_pct"].clip(lower=0.0, upper=1.0)
    return df

# Carga + label + features
edu = load_rendimiento_clean('./datasets/csvClear')
edu = build_label_aprobacion(edu)
edu = engineer_features_edu(edu)

# Split temporal: entrenar en años < max(AGNO), test en max(AGNO)
test_year = int(edu["AGNO"].dropna().max())
train_df = edu[edu["AGNO"] < test_year].dropna(subset=["label_aprobado"])
test_df  = edu[edu["AGNO"] == test_year].dropna(subset=["label_aprobado"])

feature_cols = ["prom_gral","asistencia_pct"]
X_train, y_train = train_df[feature_cols], train_df["label_aprobado"].astype(int)
X_test,  y_test  = test_df[feature_cols],  test_df["label_aprobado"].astype(int)

print(f"📊 Train: {X_train.shape}, Test({test_year}): {X_test.shape}, Prevalencia test: {y_test.mean():.1%}")



In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss

baseline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=1000, class_weight="balanced", random_state=42)),
])
print("🔄 Entrenando baseline (LR)...")
baseline.fit(X_train, y_train)
p_lr = baseline.predict_proba(X_test)[:,1]
print(f"✅ LR AUROC={roc_auc_score(y_test,p_lr):.4f} AUPRC={average_precision_score(y_test,p_lr):.4f} Brier={brier_score_loss(y_test,p_lr):.4f}")

# XGBoost
from xgboost import XGBClassifier
pos = (y_train==1).sum(); neg = (y_train==0).sum()
spw = float(neg/pos) if pos>0 else 1.0

xgb_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("clf", XGBClassifier(
        n_estimators=300, max_depth=4, learning_rate=0.06,
        subsample=0.9, colsample_bytree=0.9,
        scale_pos_weight=spw, eval_metric="logloss", random_state=42
    ))
])
print("🔄 Entrenando XGBoost...")
xgb_pipe.fit(X_train, y_train)
p_xgb = xgb_pipe.predict_proba(X_test)[:,1]
print(f"✅ XGB AUROC={roc_auc_score(y_test,p_xgb):.4f} AUPRC={average_precision_score(y_test,p_xgb):.4f} Brier={brier_score_loss(y_test,p_xgb):.4f}")

# Guardar mejor modelo
import joblib
best_pipe, best_proba = (xgb_pipe, p_xgb) if roc_auc_score(y_test,p_xgb) >= roc_auc_score(y_test,p_lr) else (baseline, p_lr)
joblib.dump(best_pipe, "model_edu.pkl")
joblib.dump(feature_cols, "feature_names_edu.pkl")
print("💾 Modelo educativo guardado: model_edu.pkl")

# Fairness simple por AGNO (temporal)
res = []
for ag in sorted(edu["AGNO"].dropna().unique()):
    m = test_df["AGNO"]==ag if ag==test_year else train_df["AGNO"]==ag
    if m.sum()>50:
        Xg = (test_df if ag==test_year else train_df).loc[m, feature_cols]
        yg = (test_df if ag==test_year else train_df).loc[m, "label_aprobado"].astype(int)
        pg = best_pipe.predict_proba(Xg)[:,1]
        res.append({"AGNO": int(ag),
                    "n": int(m.sum()),
                    "auroc": float(roc_auc_score(yg, pg)),
                    "brier": float(brier_score_loss(yg, pg))})
fair_df = pd.DataFrame(res).sort_values("AGNO")
print("\n📊 Fairness por AGNO:")
print(fair_df.to_string(index=False))
fair_df.to_csv("fairness_edu_by_year.csv", index=False)
print("💾 fairness_edu_by_year.csv escrito")
# ...existing code...